In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import sem

In [ ]:
import importlib
import load  # your module
importlib.reload(load)

from load import load_cdom_412nm_data

In [ ]:
full_data = load_cdom_412nm_data()
full_data

In [ ]:
full_data['Date'] = pd.to_datetime(full_data['Date'], errors='coerce')
full_data['Month'] = full_data['Date'].dt.month

def assign_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Other'

full_data['Season'] = full_data['Month'].apply(assign_season)


In [ ]:
season_stats = full_data.groupby('Season')['value'].agg(
    mean='mean',
    stderr=sem,
    count='count',
    q25=lambda x: x.quantile(0.25),
    q75=lambda x: x.quantile(0.75)
).reset_index()

print(season_stats)


In [ ]:

# Only include defined seasons
plot_data = full_data[full_data['Season'].isin(['Spring', 'Summer', 'Fall'])]

plt.figure(figsize=(8, 6))
sns.boxplot(x='Season', y='value', data=plot_data, palette='Set2')
plt.title('CDOM at ~412nm by Season')
plt.ylabel('Absorbance Value')
plt.xlabel('Season')
plt.ylim(-2, 2)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean

# Assuming 'lon' and 'lat' columns exist in full_data
# Filter data to include only the seasons of interest and valid lat/lon
plot_data = full_data[full_data['Season'].isin(['Spring', 'Summer', 'Fall'])].dropna(subset=['Lat', 'Lon'])

seasons = ['Spring', 'Summer', 'Fall']
fig, axes = plt.subplots(1, 3, figsize=(24, 6), subplot_kw={'projection': ccrs.PlateCarree()})

# Find global min and max for consistent color scale
vmin = -2
vmax = 2

for i, season in enumerate(seasons):
    ax = axes[i]
    season_data = plot_data[plot_data['Season'] == season]
    
    ax.set_extent([-94, -88, 27.5, 30.5], crs=ccrs.PlateCarree())
    
    # Add map features
    ax.add_feature(cfeature.LAND, zorder=1, edgecolor='black')
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

    # Scatter plot
    if not season_data.empty:
        scatter = ax.scatter(season_data['Lon'], season_data['Lat'],
                             c=season_data['value'],
                             cmap=cmocean.cm.matter,
                             vmin=vmin, vmax=vmax,
                             transform=ccrs.PlateCarree(),
                             zorder=2)
    
    ax.set_title(f'CDOM at ~412nm - {season}')

# Add a single colorbar for all subplots
fig.colorbar(scatter, ax=axes, orientation='horizontal', label='Absorbance Value', pad=0.1, aspect=40)

plt.suptitle('CDOM Recordings by Season in the Gulf of Mexico', fontsize=16, y=0.95)
plt.show()